In [1]:
import numpy as np
import matplotlib.pyplot as plt
from OC_NN_model import Model
from sklearn import svm
from sklearn.metrics import roc_curve, auc

import torch
import torch.nn as nn
import torch.utils as utils

from torchvision import datasets, transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset_name = 'train_data9'
train_set = datasets.ImageFolder(root='dataset/preproced_data/%s/'%dataset_name,
                       transform=transforms.Compose([
#                              transforms.RandomHorizontalFlip(p=0.5),
                         transforms.Grayscale(),
                         transforms.Resize((64,64)),
                         transforms.ToTensor(),
                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
train_loader = utils.data.DataLoader(dataset=train_set, batch_size=1000,shuffle=True)

for idx, (data, target) in enumerate(train_loader):
    x_train = data.cpu().numpy()

score_num = 1
dir_name = ['abnormal', 'normal']
test_set = datasets.ImageFolder(root='dataset/dongdong2/%s/'%dir_name[score_num],
                           transform=transforms.Compose([
#                              transforms.RandomHorizontalFlip(p=0.5),
                             transforms.Grayscale(),
                             transforms.Resize((64,64)),
                             transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
test_loader = utils.data.DataLoader(dataset=test_set,batch_size=100,shuffle=False)

for idx, (data, target) in enumerate(test_loader):
    x_test = data.cpu().numpy()
    
score_num = 0    
test_set = datasets.ImageFolder(root='dataset/dongdong2/%s/'%dir_name[score_num],
                           transform=transforms.Compose([
#                              transforms.RandomHorizontalFlip(p=0.5),
                             transforms.Grayscale(),
                             transforms.Resize((64,64)),
                             transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
test_loader = utils.data.DataLoader(dataset=test_set,batch_size=100,shuffle=False)

for idx, (data, target) in enumerate(test_loader):
    x_out = data.cpu().numpy()

x_train = np.reshape(x_train, (len(x_train),-1))
x_test = np.reshape(x_test, (len(x_test), -1))
x_out = np.reshape(x_out, (len(x_out),-1))

x_train = torch.Tensor(x_train).to(device)
x_test = torch.Tensor(x_test).to(device)
x_out = torch.Tensor(x_out).to(device)

print(np.shape(x_train), np.shape(x_test), np.shape(x_out))

torch.Size([631, 4096]) torch.Size([25, 4096]) torch.Size([31, 4096])


In [16]:
np.random.seed(37)
torch.manual_seed(37)
torch.cuda.manual_seed_all(37)
torch.backends.cudnn.deterministic = True

nu = 0.95
learning_rate = 0.0005
model = nn.DataParallel(Model(x_train.size()[-1], 512)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

mb_div = 1
mb_idx = int(len(x_train)/mb_div)
s = np.arange(len(x_train))

for i in range(10000):
    np.random.shuffle(s)
    x_train = x_train[s]
    
    model.train()
    for mb in range(mb_div):
        dx_train = x_train[mb*mb_idx:(mb+1)*mb_idx]
        
        optimizer.zero_grad()
        output, loss, r = model(dx_train, nu=nu)
        loss = torch.mean(loss)
        loss.backward()
        optimizer.step()
        
        output = output.cpu().detach().numpy()
#         r = np.quantile(output, nu)
        
    if i %100==0:
        model.eval()
        output, _, r = model(x_train, nu=nu)
        out_test, _, r = model(x_test, nu=nu)
        out_out, _, r = model(x_out, nu=nu)
        
        output = output.cpu().detach().numpy()
        out_test = out_test.cpu().detach().numpy()
        out_out = out_out.cpu().detach().numpy()
        r = np.mean(r.cpu().detach().numpy())
        print(r, np.mean(r<output), np.mean(r<out_test), np.mean(r>out_out))

0.9995 1.0 1.0 0.0
0.9920362 1.0 1.0 0.0
0.985902 1.0 1.0 0.0
0.9746901 1.0 1.0 0.0
0.963098 1.0 1.0 0.0
0.9521996 1.0 1.0 0.0
0.9415015 0.0 0.0 1.0
0.9302339 1.0 1.0 0.0
0.91954374 1.0 1.0 0.0
0.9088884 1.0 1.0 0.0
0.89856005 1.0 1.0 0.0
0.88836443 1.0 1.0 0.0
0.87827945 1.0 1.0 0.0
0.86827064 1.0 1.0 0.0
0.85829306 1.0 1.0 0.0
0.8478008 1.0 1.0 0.0
0.83782554 1.0 1.0 0.0
0.82767963 1.0 1.0 0.0
0.8182502 1.0 1.0 0.0
0.8078532 1.0 1.0 0.0
0.797829 1.0 1.0 0.0
0.78772885 1.0 1.0 0.0
0.77788097 1.0 1.0 0.0
0.76795334 1.0 1.0 0.0
0.7579924 1.0 1.0 0.0
0.748153 1.0 1.0 0.0
0.73836756 1.0 1.0 0.0
0.7286766 1.0 1.0 0.0
0.71893346 1.0 1.0 0.0
0.7092571 1.0 1.0 0.0
0.6999637 1.0 1.0 0.0
0.68997693 1.0 1.0 0.0
0.68005407 1.0 1.0 0.0
0.6701384 1.0 1.0 0.0
0.660378 1.0 1.0 0.0
0.65062666 1.0 1.0 0.0
0.64120835 1.0 1.0 0.0
0.631868 1.0 1.0 0.0
0.62189585 1.0 1.0 0.0
0.6120848 1.0 1.0 0.0
0.6023112 1.0 1.0 0.0
0.5928917 1.0 1.0 0.0
0.58370495 1.0 1.0 0.0
0.57369053 1.0 1.0 0.0
0.5638383 1.0 1.0 0.0

KeyboardInterrupt: 

In [ ]:
# def cal_AUC(sensitivity, specificity):
#     wid = (1-specificity)[:-1]-(1-specificity)[1:]
#     auc = np.sum(sensitivity[1:]*wid)
#     return auc

# fpr_list = []
# tpr_list = []
# for k in range(1,21):
#     clf = svm.OneClassSVM(nu=0.05*k, kernel='rbf', gamma='auto')
#     clf.fit(x_train)
#     y_pred_train = clf.predict(x_train)
#     y_pred_test = clf.predict(x_test)
#     y_pred_out = clf.predict(x_out)
#     print(y_pred_test)
    
#     label = np.append(np.zeros(len(y_pred_test)),np.ones(len(y_pred_out)))
#     y_hat = np.append(np.float32(y_pred_test!=1), np.float32(y_pred_out!=1))
    
#     fpr, tpr, threshold = roc_curve(label, y_hat, drop_intermediate=False)
#     fpr_list.append(fpr[1])
#     tpr_list.append(tpr[1])

# auc = cal_AUC(np.array(tpr_list), np.array(fpr_list))
# plt.plot(fpr_list, tpr_list, 'ro-')
# plt.xlabel('1-Specificity  (FPR)', fontsize=12)
# plt.ylabel('Sensitivity  (TPR)', fontsize=12)
# plt.plot([0,1],[0,1],'k--')
# plt.title('AUC : %.4f'%auc, fontsize=14)
# plt.savefig('ROC_plot.png')
# plt.show()